This notebook grabs census data from the census website. The Tracts section has functions that ultimately lets you input a list of states you are interested in and the function tract_level_df grabs the data. We have census data that we are interested in, ACS data that we are interested in, and also want our data to be in percents so that we can run the ecological inference on it. 

The Counties section isn't quite working. As of right now, we are getting one more ACS county than census county, which makes merging the data together into one dataframe impossible. One potential workaround is using Rebecca and Odelia's notebooks to aggregate the ACS data by county and just not have the census data in that comparison. 

In [7]:
import requests
import pandas as pd

# Counties

In [21]:
def fetch_data_c(base_url, var_map):
    params = {
        "get": ",".join(var_map.keys()),
        "for": "county:*",
        "in": "state:*",
        "key": 'df8c76e570e47e69b0d4523fbde247bc091d81dc'
    }
    
    raise_count = 0
    success = False
    while not success and raise_count < 5:
        try:
            # Set a timeout of 5 minutes
            r = requests.get(base_url, params=params, timeout=300)
            r.raise_for_status()
            success = True
        except Exception as e:
            if raise_count >= 5:
                print(f"Failed to fetch data after 5 attempts, aborting.")
                raise e
            else:
                raise_count += 1
                print(f"Error fetching data, retrying {raise_count} of 5", end="\r")
                # Sleep for 5 seconds and try again
                sleep(5)

            
    data = r.json()
    df = pd.DataFrame(data[1:], columns=data[0])
    df["GEOID"] = df["state"].astype(str) + df["county"].astype(str)
    df = df.rename(columns=var_map)
    return df

In [30]:
def county_level_df():
    # Input your Census API key
    API_KEY = "df8c76e570e47e69b0d4523fbde247bc091d81dc"
    
    # Geographic scope: all tracts in Massachusetts
    #state_fips = inputted_state_fips
    county = "*"  # all counties
    geography = "tract:*"
    
    # Define variables to pull
    pl_vars = {
        "P1_001N": "totpop_PL", #these have been checked
        "P2_002N": "hispanic_PL", 
        "P1_002N": "white_PL",
        "P1_004N": "black_PL",
        "P1_008N": "sor_PL"
    }
    
    acs_vars = {
        "B03002_001E": "totpop_ACS", #good
        "B03002_012E": "hispanic_ACS", #good
        "B02001_002E": "white_ACS", 
        "B02001_003E": "black_ACS",
        "B02001_007E": "sor_ACS",
        "B04004_022E": "brazilian_ACS",
        "B04004_045E": "guyanese_ACS",
        "B04004_074E": "cabo_verdean_ACS",
        "B04004_097E": "belizean_ACS", #everything above here is correct 
        #"B04004_002E": "algerian_ACS",
        "B04004_016E": "armenian_ACS",
        #"B04004_006E": "azerbaijani_ACS", #WRONG
        #"B04004_007E": "bahraini_ACS", #WRONG
        "B04004_030E": "cypriot_ACS",
        #"B04004_015E": "djiboutian_ACS",
        "B04004_007E": "egyptian_ACS",
        #"B04004_032E": "georgian_ACS",
        "B04004_048E": "iranian_ACS",
        "B04004_008E": "iraqi_ACS",
        "B04004_050E": "israeli_ACS",
        "B04004_009E": "jordanian_ACS",
        #"B04004_044E": "kuwaiti_ACS",
        "B04004_010E": "lebanese_ACS",
        #"B04004_047E": "libyan_ACS",
        "B04004_056E": "maltese_ACS",
        #"B04004_049E": "mauritanian_ACS",
        "B04004_011E": "moroccan_ACS",
        #"B04004_052E": "omani_ACS",
        "B04004_012E": "palestinian_ACS",
        #"B04004_055E": "qatari_ACS",
        #"B04004_062E": "saudi_ACS",
        "B04004_084E": "sudanese_ACS",
        "B04004_013E": "syrian_ACS",
        #"B04004_079E": "sahrawi_ACS",
        #"B04004_080E": "tunisian_ACS",
        "B04004_091E": "turkish_ACS",
        #"B04004_092E": "emirati_ACS",
        #"B04004_093E": "yemeni_ACS" #below are ones that we have added
        "B04004_006E": "arab_ACS", #this is an umbrella for some categories that we already have above
        "B04004_014E": "arab_arab_ACS",
        "B04004_015E": "arab_other_ACS",
        "B04004_017E": "chaldean_ACS" #assyrian/chaldean/syriac
    }
    
    
    # Fetch PL and ACS data
    pl_df = fetch_data_c("https://api.census.gov/data/2020/dec/pl", pl_vars)
    acs_df = fetch_data_c("https://api.census.gov/data/2022/acs/acs5", acs_vars)
    
    assert set(pl_df["GEOID"]) == set(acs_df["GEOID"]), "GEOIDs in PL and ACS data do not match."
    
    # Merge on GEOID
    merged_df = pd.merge(pl_df, acs_df, on=["GEOID", "state", "county", "tract"], how="inner")
    
    # Something went horribly wrong if this throws
    assert pl_df.shape[0] == merged_df.shape[0] and merged_df.shape[0] == acs_df.shape[0], "Number of rows in PL and ACS data do not match after merge."
    
    # Convert numeric columns
    cols_to_numeric = list(pl_vars.values()) + list(acs_vars.values())
    merged_df[cols_to_numeric] = merged_df[cols_to_numeric].apply(pd.to_numeric, errors="coerce")
    
    
    # World Bank MENA
    mena_world_bank = [
             "egyptian_ACS",
            "iranian_ACS", "iraqi_ACS", "israeli_ACS", "jordanian_ACS", 
            "lebanese_ACS", "maltese_ACS", "moroccan_ACS",
            "palestinian_ACS", "syrian_ACS", "arab_other_ACS", "arab_arab_ACS" #we added this 
        ]#"algerian_ACS", "bahraini_ACS","djiboutian_ACS","kuwaiti_ACS","libyan_ACS", "omani_ACS","qatari_ACS","saudi_ACS","tunisian_ACS","emirati_ACS", "yemeni_ACS"
    merged_df["mena_world_bank_ACS"] = merged_df[mena_world_bank].sum(axis=1)
    # UNHCR MENA
    mena_unhcr = [
             "egyptian_ACS", "iraqi_ACS", "jordanian_ACS",
             "lebanese_ACS", "maltese_ACS",
            "moroccan_ACS", "palestinian_ACS",
            "syrian_ACS", "arab_other_ACS", "arab_arab_ACS" #added last two
        ] #"algerian_ACS", "bahraini_ACS","kuwaiti_ACS","libyan_ACS", "mauritanian_ACS", "omani_ACS","qatari_ACS", "saudi_ACS",, "tunisian_ACS", "emirati_ACS", "yemeni_ACS"
    merged_df["mena_unhcr_ACS"] = merged_df[mena_unhcr].sum(axis=1)
    # UNSD MENA
    mena_unsd = [
             "armenian_ACS", 
            "cypriot_ACS", "egyptian_ACS", "iraqi_ACS", "israeli_ACS",
            "jordanian_ACS", "lebanese_ACS", "moroccan_ACS",
             "palestinian_ACS", "sudanese_ACS",
            "syrian_ACS", "turkish_ACS", "arab_other_ACS", "arab_arab_ACS" #added last two
        ]#"algerian_ACS", "azerbaijani_ACS","bahraini_ACS","georgian_ACS","kuwaiti_ACS","libyan_ACS", "omani_ACS","qatari_ACS", "saudi_ACS","tunisian_ACS","emirati_ACS", "sahrawi_ACS","yemeni_ACS"
    merged_df["mena_unsd_ACS"] = merged_df[mena_unsd].sum(axis=1)
    return merged_df

# Tracts

In [103]:
def fetch_data(base_url, var_map):
    params = {
        "get": ",".join(var_map.keys()),
        "for": "tract:*",
        "in": f"state:{state_fips} county:{county}",
        "key": API_KEY
    }
    
    raise_count = 0
    success = False
    while not success and raise_count < 5:
        try:
            # Set a timeout of 5 minutes
            r = requests.get(base_url, params=params, timeout=300)
            r.raise_for_status()
            success = True
        except Exception as e:
            if raise_count >= 5:
                print(f"Failed to fetch data after 5 attempts, aborting.")
                raise e
            else:
                raise_count += 1
                print(f"Error fetching data, retrying {raise_count} of 5", end="\r")
                # Sleep for 5 seconds and try again
                sleep(5)

            
    data = r.json()
    df = pd.DataFrame(data[1:], columns=data[0])
    df["GEOID"] = df["state"].astype(str) + df["county"].astype(str) + df["tract"].astype(str)
    df = df.rename(columns=var_map)
    return df

In [163]:
def tract_level_df(inputted_state_fips):
    # Input your Census API key
    API_KEY = "df8c76e570e47e69b0d4523fbde247bc091d81dc"
    
    # Geographic scope: all tracts in Massachusetts
    state_fips = inputted_state_fips
    county = "*"  # all counties
    geography = "tract:*"
    
    # Define variables to pull
    pl_vars = {
        "P1_001N": "totpop_PL", #these have been checked
        "P2_002N": "hispanic_PL", 
        "P1_002N": "white_PL",
        "P1_004N": "black_PL",
        "P1_008N": "sor_PL"
    }
    
    acs_vars = {
        "B03002_001E": "totpop_ACS", #good
        "B03002_012E": "hispanic_ACS", #good
        "B02001_002E": "white_ACS", 
        "B02001_003E": "black_ACS",
        "B02001_007E": "sor_ACS",
        "B04004_022E": "brazilian_ACS",
        "B04004_045E": "guyanese_ACS",
        "B04004_074E": "cabo_verdean_ACS",
        "B04004_097E": "belizean_ACS", #everything above here is correct 
        #"B04004_002E": "algerian_ACS",
        "B04004_016E": "armenian_ACS",
        #"B04004_006E": "azerbaijani_ACS", #WRONG
        #"B04004_007E": "bahraini_ACS", #WRONG
        "B04004_030E": "cypriot_ACS",
        #"B04004_015E": "djiboutian_ACS",
        "B04004_007E": "egyptian_ACS",
        #"B04004_032E": "georgian_ACS",
        "B04004_048E": "iranian_ACS",
        "B04004_008E": "iraqi_ACS",
        "B04004_050E": "israeli_ACS",
        "B04004_009E": "jordanian_ACS",
        #"B04004_044E": "kuwaiti_ACS",
        "B04004_010E": "lebanese_ACS",
        #"B04004_047E": "libyan_ACS",
        "B04004_056E": "maltese_ACS",
        #"B04004_049E": "mauritanian_ACS",
        "B04004_011E": "moroccan_ACS",
        #"B04004_052E": "omani_ACS",
        "B04004_012E": "palestinian_ACS",
        #"B04004_055E": "qatari_ACS",
        #"B04004_062E": "saudi_ACS",
        "B04004_084E": "sudanese_ACS",
        "B04004_013E": "syrian_ACS",
        #"B04004_079E": "sahrawi_ACS",
        #"B04004_080E": "tunisian_ACS",
        "B04004_091E": "turkish_ACS",
        #"B04004_092E": "emirati_ACS",
        #"B04004_093E": "yemeni_ACS" #below are ones that we have added
        "B04004_006E": "arab_ACS", #this is an umbrella for some categories that we already have above
        "B04004_014E": "arab_arab_ACS",
        "B04004_015E": "arab_other_ACS",
        "B04004_017E": "chaldean_ACS" #assyrian/chaldean/syriac
    }
    
    
    # Fetch PL and ACS data
    pl_df = fetch_data("https://api.census.gov/data/2020/dec/pl", pl_vars)
    acs_df = fetch_data("https://api.census.gov/data/2022/acs/acs5", acs_vars)
    
    assert set(pl_df["GEOID"]) == set(acs_df["GEOID"]), "GEOIDs in PL and ACS data do not match."
    
    # Merge on GEOID
    merged_df = pd.merge(pl_df, acs_df, on=["GEOID", "state", "county", "tract"], how="inner")
    
    # Something went horribly wrong if this throws
    assert pl_df.shape[0] == merged_df.shape[0] and merged_df.shape[0] == acs_df.shape[0], "Number of rows in PL and ACS data do not match after merge."
    
    # Convert numeric columns
    cols_to_numeric = list(pl_vars.values()) + list(acs_vars.values())
    merged_df[cols_to_numeric] = merged_df[cols_to_numeric].apply(pd.to_numeric, errors="coerce")
    
    
    # World Bank MENA
    mena_world_bank = [
             "egyptian_ACS",
            "iranian_ACS", "iraqi_ACS", "israeli_ACS", "jordanian_ACS", 
            "lebanese_ACS", "maltese_ACS", "moroccan_ACS",
            "palestinian_ACS", "syrian_ACS", "arab_other_ACS", "arab_arab_ACS" #we added this 
        ]#"algerian_ACS", "bahraini_ACS","djiboutian_ACS","kuwaiti_ACS","libyan_ACS", "omani_ACS","qatari_ACS","saudi_ACS","tunisian_ACS","emirati_ACS", "yemeni_ACS"
    merged_df["mena_world_bank_ACS"] = merged_df[mena_world_bank].sum(axis=1)
    # UNHCR MENA
    mena_unhcr = [
             "egyptian_ACS", "iraqi_ACS", "jordanian_ACS",
             "lebanese_ACS", "maltese_ACS",
            "moroccan_ACS", "palestinian_ACS",
            "syrian_ACS", "arab_other_ACS", "arab_arab_ACS" #added last two
        ] #"algerian_ACS", "bahraini_ACS","kuwaiti_ACS","libyan_ACS", "mauritanian_ACS", "omani_ACS","qatari_ACS", "saudi_ACS",, "tunisian_ACS", "emirati_ACS", "yemeni_ACS"
    merged_df["mena_unhcr_ACS"] = merged_df[mena_unhcr].sum(axis=1)
    # UNSD MENA
    mena_unsd = [
             "armenian_ACS", 
            "cypriot_ACS", "egyptian_ACS", "iraqi_ACS", "israeli_ACS",
            "jordanian_ACS", "lebanese_ACS", "moroccan_ACS",
             "palestinian_ACS", "sudanese_ACS",
            "syrian_ACS", "turkish_ACS", "arab_other_ACS", "arab_arab_ACS" #added last two
        ]#"algerian_ACS", "azerbaijani_ACS","bahraini_ACS","georgian_ACS","kuwaiti_ACS","libyan_ACS", "omani_ACS","qatari_ACS", "saudi_ACS","tunisian_ACS","emirati_ACS", "sahrawi_ACS","yemeni_ACS"
    merged_df["mena_unsd_ACS"] = merged_df[mena_unsd].sum(axis=1)
    return merged_df

In [152]:
def make_percents_df(df, total_column, numerator_column):
    column_name = numerator_column + "_pct"
    df[column_name] = df[numerator_column]/df[total_column]
    return df

In [165]:
states_fips = [
    "06", # California
    "25", # Massachusetts
    "26", # Michigan
    "36", # New York
    "48" #Texas
]

In [172]:
for state_fips in states_fips:
    df = tract_level_df(state_fips)
    columns_to_percent = [ #this will be the columns of the dataframe that we will want to find percentages of. you need to change this if you want to look at different races/ancestries, etc
         'hispanic_PL',
         'white_PL',
         'black_PL',
         'sor_PL',
         'hispanic_ACS',
         'white_ACS',
         'black_ACS',
         'sor_ACS',
         'brazilian_ACS',
         'guyanese_ACS',
         'cabo_verdean_ACS',
         'belizean_ACS',
         #'algerian_ACS',
         'armenian_ACS',
         #'azerbaijani_ACS',
         #'bahraini_ACS',
         'cypriot_ACS',
         #'djiboutian_ACS',
         'egyptian_ACS',
         #'georgian_ACS',
         'iranian_ACS',
         'iraqi_ACS',
         'israeli_ACS',
         'jordanian_ACS',
         #'kuwaiti_ACS',
         'lebanese_ACS',
         #'libyan_ACS',
         'maltese_ACS',
         #'mauritanian_ACS',
         'moroccan_ACS',
         #'omani_ACS',
         'palestinian_ACS',
         #'qatari_ACS',
         #'saudi_ACS',
         'sudanese_ACS',
         'syrian_ACS',
         #'sahrawi_ACS',
         #'tunisian_ACS',
         'turkish_ACS',
         #'emirati_ACS',
         #'yemeni_ACS',
        "arab_ACS", #this is an umbrella for some categories that we already have above
        "arab_arab_ACS",
         "arab_other_ACS",
         "chaldean_ACS", #assyrian/chaldean/syriac
         'mena_world_bank_ACS',
         'mena_unhcr_ACS',
         'mena_unsd_ACS'
    ]
    for col in columns_to_percent:
        if col[-2:] == "PL":
            df = make_percents_df(df, 'totpop_PL', col)
        else:
            df = make_percents_df(df, 'totpop_ACS', col)
    #print(df)
    file_name = 'ecologicalcsvs/' + state_fips + 'ecological_acs_pl.csv'
    df.to_csv(file_name)

# Chloe's Work

In [40]:
pl_vars = {
    "P1_001N": "totpop_PL", #these have been checked
    "P2_002N": "hispanic_PL", 
    "P1_002N": "white_PL",
    "P1_004N": "black_PL",
    "P1_008N": "sor_PL"
}

acs_vars = {
    "B03002_001E": "totpop_ACS", #good
    "B03002_012E": "hispanic_ACS", #good
    "B02001_002E": "white_ACS", 
    "B02001_003E": "black_ACS",
    "B02001_007E": "sor_ACS",
    "B04004_022E": "brazilian_ACS",
    "B04004_045E": "guyanese_ACS",
    "B04004_074E": "cabo_verdean_ACS",
    "B04004_097E": "belizean_ACS", #everything above here is correct 
    #"B04004_002E": "algerian_ACS",
    "B04004_016E": "armenian_ACS",
    #"B04004_006E": "azerbaijani_ACS", #WRONG
    #"B04004_007E": "bahraini_ACS", #WRONG
    "B04004_030E": "cypriot_ACS",
    #"B04004_015E": "djiboutian_ACS",
    "B04004_007E": "egyptian_ACS",
    #"B04004_032E": "georgian_ACS",
    "B04004_048E": "iranian_ACS",
    "B04004_008E": "iraqi_ACS",
    "B04004_050E": "israeli_ACS",
    "B04004_009E": "jordanian_ACS",
    #"B04004_044E": "kuwaiti_ACS",
    "B04004_010E": "lebanese_ACS",
    #"B04004_047E": "libyan_ACS",
    "B04004_056E": "maltese_ACS",
    #"B04004_049E": "mauritanian_ACS",
    "B04004_011E": "moroccan_ACS",
    #"B04004_052E": "omani_ACS",
    "B04004_012E": "palestinian_ACS",
    #"B04004_055E": "qatari_ACS",
    #"B04004_062E": "saudi_ACS",
    "B04004_084E": "sudanese_ACS",
    "B04004_013E": "syrian_ACS",
    #"B04004_079E": "sahrawi_ACS",
    #"B04004_080E": "tunisian_ACS",
    "B04004_091E": "turkish_ACS",
    #"B04004_092E": "emirati_ACS",
    #"B04004_093E": "yemeni_ACS" #below are ones that we have added
    "B04004_006E": "arab_ACS", #this is an umbrella for some categories that we already have above
    "B04004_014E": "arab_arab_ACS",
    "B04004_015E": "arab_other_ACS",
    "B04004_017E": "chaldean_ACS" #assyrian/chaldean/syriac
}
pl_df = fetch_data_c("https://api.census.gov/data/2020/dec/pl", pl_vars)
acs_df = fetch_data_c("https://api.census.gov/data/2022/acs/acs5", acs_vars)

In [56]:
pl_df

,totpop_PL,hispanic_PL,white_PL,black_PL,sor_PL,state,county,GEOID
0,58805,2117,55648,11445,910,01,001,01001
1,231767,12686,216743,18217,5335,01,003,01003
2,25223,1510,24523,11933,1039,01,005,01005
3,22293,740,21534,4413,465,01,007,01007
4,59134,5771,55478,845,3431,01,009,01009
...,...,...,...,...,...,...,...,...
3216,14723,1319,13925,69,496,27,129,27129
3217,15331,238,14626,87,128,27,135,27135
3218,97183,2820,92120,3666,1189,27,141,27141
3219,37406,3140,35421,1373,1413,27,147,27147


In [58]:
acs_df

,totpop_ACS,hispanic_ACS,white_ACS,black_ACS,sor_ACS,brazilian_ACS,guyanese_ACS,cabo_verdean_ACS,belizean_ACS,armenian_ACS,...,sudanese_ACS,syrian_ACS,turkish_ACS,arab_ACS,arab_arab_ACS,arab_other_ACS,chaldean_ACS,state,county,GEOID
0,58761,1864,43747,11496,321,0,0,0,0,15,...,0,0,0,0,0,0,0,01,001,01001
1,233420,11210,195998,19445,4414,434,0,0,0,0,...,0,3,47,79,0,0,0,01,003,01003
2,24877,1202,11309,11668,1088,0,0,0,0,0,...,0,0,0,5,0,0,0,01,005,01005
3,22251,650,16872,4603,108,0,0,0,0,0,...,0,0,0,0,0,0,0,01,007,01007
4,59077,5721,53941,729,1859,0,0,0,0,0,...,0,0,0,3,0,0,0,01,009,01009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3217,54182,53023,21954,2444,9811,0,0,0,0,0,...,0,0,0,20,20,0,0,72,145,72145
3218,8199,7744,1813,579,5234,0,0,0,0,0,...,0,0,0,0,0,0,0,72,147,72147
3219,21984,21905,8732,1889,2868,0,0,0,0,0,...,0,0,0,0,0,0,0,72,149,72149
3220,30313,30252,2393,11349,15593,0,0,0,0,0,...,0,0,0,0,0,0,0,72,151,72151


In [65]:
print(set(pl_df["GEOID"]).difference(set(acs_df["GEOID"])))



{'09007', '09003', '09009', '09011', '09015', '09013', '09001', '09005'}


In [50]:
pl_df.to_csv('pl_df_july18.csv')

In [52]:
acs_df.to_csv('acs_df_july18.csv')

In [96]:
# Input your Census API key
API_KEY = "df8c76e570e47e69b0d4523fbde247bc091d81dc"

# Geographic scope: all tracts in Massachusetts
state_fips = "06"
county = "*"  # all counties
geography = "tract:*"

# Define variables to pull
pl_vars = {
    "P1_001N": "totpop_PL", #these have been checked
    "P2_002N": "hispanic_PL", 
    "P1_002N": "white_PL",
    "P1_004N": "black_PL",
    "P1_008N": "sor_PL"
}

acs_vars = {
    "B03002_001E": "totpop_ACS", #good
    "B03002_012E": "hispanic_ACS", #good
    "B02001_002E": "white_ACS", 
    "B02001_003E": "black_ACS",
    "B02001_007E": "sor_ACS",
    "B04006_022E": "brazilian_ACS",
    "B04006_045E": "guyanese_ACS",
    "B04006_074E": "cabo_verdean_ACS",
    "B04006_097E": "belizean_ACS", #fixed 
    "B04006_002E": "algerian_ACS",
    "B04006_004E": "armenian_ACS",
    "B04006_006E": "azerbaijani_ACS",
    "B04006_007E": "bahraini_ACS",
    "B04006_011E": "cypriot_ACS",
    "B04006_015E": "djiboutian_ACS",
    "B04006_028E": "egyptian_ACS",
    "B04006_032E": "georgian_ACS",
    "B04006_039E": "iranian_ACS",
    "B04006_040E": "iraqi_ACS",
    "B04006_041E": "israeli_ACS",
    "B04006_042E": "jordanian_ACS",
    "B04006_044E": "kuwaiti_ACS",
    "B04006_046E": "lebanese_ACS",
    "B04006_047E": "libyan_ACS",
    "B04006_048E": "maltese_ACS",
    "B04006_049E": "mauritanian_ACS",
    "B04006_050E": "moroccan_ACS",
    "B04006_052E": "omani_ACS",
    "B04006_054E": "palestinian_ACS",
    "B04006_055E": "qatari_ACS",
    "B04006_062E": "saudi_ACS",
    "B04006_077E": "sudanese_ACS",
    "B04006_078E": "syrian_ACS",
    "B04006_079E": "sahrawi_ACS",
    "B04006_080E": "tunisian_ACS",
    "B04006_088E": "turkish_ACS",
    "B04006_092E": "emirati_ACS",
    "B04006_093E": "yemeni_ACS"
}

mena_world_bank_ACS = [
    acs_vars["B04006_002E"],  # Algerian
    acs_vars["B04006_007E"],  # Bahraini
    acs_vars["B04006_015E"],  # Djiboutian
    acs_vars["B04006_028E"],  # Egyptian
    acs_vars["B04006_039E"],  # Iranian
    acs_vars["B04006_040E"],  # Iraqi
    acs_vars["B04006_041E"],  # Israeli
    acs_vars["B04006_042E"],  # Jordanian
    acs_vars["B04006_044E"],  # Kuwaiti
    acs_vars["B04006_046E"],  # Lebanese
    acs_vars["B04006_047E"],  # Libyan
    acs_vars["B04006_048E"],  # Maltese
    acs_vars["B04006_050E"],  # Moroccan
    acs_vars["B04006_052E"],  # Omani
    acs_vars["B04006_054E"],  # Palestinian
    acs_vars["B04006_055E"],  # Qatari
    acs_vars["B04006_062E"],  # Saudi
    acs_vars["B04006_078E"],  # Syrian
    acs_vars["B04006_080E"],  # Tunisian
    acs_vars["B04006_092E"],  # Emirati
    acs_vars["B04006_093E"],  # Yemeni
]


def fetch_data(base_url, var_map):
    params = {
        "get": ",".join(var_map.keys()),
        "for": "tract:*",
        "in": f"state:{state_fips} county:{county}",
        "key": API_KEY
    }
    
    raise_count = 0
    success = False
    while not success and raise_count < 5:
        try:
            # Set a timeout of 5 minutes
            r = requests.get(base_url, params=params, timeout=300)
            r.raise_for_status()
            success = True
        except Exception as e:
            if raise_count >= 5:
                print(f"Failed to fetch data after 5 attempts, aborting.")
                raise e
            else:
                raise_count += 1
                print(f"Error fetching data, retrying {raise_count} of 5", end="\r")
                # Sleep for 5 seconds and try again
                sleep(5)

            
    data = r.json()
    df = pd.DataFrame(data[1:], columns=data[0])
    df["GEOID"] = df["state"].astype(str) + df["county"].astype(str) + df["tract"].astype(str)
    df = df.rename(columns=var_map)
    return df

# Fetch PL and ACS data
pl_df = fetch_data("https://api.census.gov/data/2020/dec/pl", pl_vars)
acs_df = fetch_data("https://api.census.gov/data/2022/acs/acs5", acs_vars)

assert set(pl_df["GEOID"]) == set(acs_df["GEOID"]), "GEOIDs in PL and ACS data do not match."

# Merge on GEOID
ca_merged_df = pd.merge(pl_df, acs_df, on=["GEOID", "state", "county", "tract"], how="inner")

# Something went horribly wrong if this throws
assert pl_df.shape[0] == ca_merged_df.shape[0] and ca_merged_df.shape[0] == acs_df.shape[0], "Number of rows in PL and ACS data do not match after merge."

# Convert numeric columns
cols_to_numeric = list(pl_vars.values()) + list(acs_vars.values())
ca_merged_df[cols_to_numeric] = ca_merged_df[cols_to_numeric].apply(pd.to_numeric, errors="coerce")


# World Bank MENA
mena_world_bank = [
        "algerian_ACS", "bahraini_ACS", "djiboutian_ACS", "egyptian_ACS",
        "iranian_ACS", "iraqi_ACS", "israeli_ACS", "jordanian_ACS", "kuwaiti_ACS",
        "lebanese_ACS", "libyan_ACS", "maltese_ACS", "moroccan_ACS", "omani_ACS",
        "palestinian_ACS", "qatari_ACS", "saudi_ACS", "syrian_ACS",
        "tunisian_ACS", "emirati_ACS", "yemeni_ACS"
    ]
ca_merged_df["mena_world_bank_ACS"] = ca_merged_df[mena_world_bank].sum(axis=1)
# UNHCR MENA
mena_unhcr = [
        "algerian_ACS", "bahraini_ACS", "egyptian_ACS", "iraqi_ACS", "jordanian_ACS",
        "kuwaiti_ACS", "lebanese_ACS", "libyan_ACS", "maltese_ACS", "mauritanian_ACS",
        "moroccan_ACS", "omani_ACS", "palestinian_ACS", "qatari_ACS", "saudi_ACS",
        "syrian_ACS", "tunisian_ACS", "emirati_ACS", "yemeni_ACS"
    ]
ca_merged_df["mena_unhcr_ACS"] = ca_merged_df[mena_unhcr].sum(axis=1)
# UNSD MENA
mena_unsd = [
        "algerian_ACS", "armenian_ACS", "azerbaijani_ACS", "bahraini_ACS",
        "cypriot_ACS", "egyptian_ACS", "georgian_ACS", "iraqi_ACS", "israeli_ACS",
        "jordanian_ACS", "kuwaiti_ACS", "lebanese_ACS", "libyan_ACS", "moroccan_ACS",
        "omani_ACS", "palestinian_ACS", "qatari_ACS", "saudi_ACS", "sudanese_ACS",
        "syrian_ACS", "tunisian_ACS", "turkish_ACS", "emirati_ACS", "sahrawi_ACS",
        "yemeni_ACS"
    ]
ca_merged_df["mena_unsd_ACS"] = ca_merged_df[mena_unsd].sum(axis=1)

In [98]:
ca_merged_df

,totpop_PL,hispanic_PL,white_PL,black_PL,sor_PL,state,county,tract,GEOID,totpop_ACS,...,sudanese_ACS,syrian_ACS,sahrawi_ACS,tunisian_ACS,turkish_ACS,emirati_ACS,yemeni_ACS,mena_world_bank_ACS,mena_unhcr_ACS,mena_unsd_ACS
0,3038,205,2645,144,80,06,001,400100,06001400100,3269,...,0,0,0,0,0,69,9,630,808,622
1,2001,207,1716,40,41,06,001,400200,06001400200,2147,...,0,0,9,0,0,47,12,620,743,615
2,5504,547,4772,561,191,06,001,400300,06001400300,5619,...,0,0,0,0,0,0,41,1078,1427,1117
3,4112,374,3595,289,122,06,001,400400,06001400400,4278,...,0,0,11,0,0,49,8,614,1003,636
4,3644,437,3086,625,168,06,001,400500,06001400500,3949,...,0,0,0,8,0,18,8,503,793,496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9124,2412,430,2003,214,72,06,115,040902,06115040902,1868,...,0,0,0,0,0,4,20,388,591,388
9125,3594,626,3205,26,358,06,115,041001,06115041001,3672,...,0,0,0,0,0,0,67,485,765,496
9126,3935,387,3562,22,147,06,115,041002,06115041002,3417,...,0,0,0,0,0,27,0,1072,1339,1072
9127,3187,254,2817,33,86,06,115,041101,06115041101,2288,...,0,0,0,0,0,0,19,597,795,579


In [ ]:


# Input your Census API key
API_KEY = "df8c76e570e47e69b0d4523fbde247bc091d81dc"

# Geographic scope: all tracts in Massachusetts
state_fips = "26" #REPLACE STATE CODE HERE
county = "*"  # all counties
geography = "tract:*"

# Define variables to pull
pl_vars = {
    "P1_001N": "totpop_PL", #these have been checked
    "P2_002N": "hispanic_PL", 
    "P1_002N": "white_PL",
    "P1_004N": "black_PL",
    "P1_008N": "sor_PL"
}

acs_vars = {
    "B03002_001E": "totpop_ACS", #good
    "B03002_012E": "hispanic_ACS", #good
    "B02001_002E": "white_ACS", 
    "B02001_003E": "black_ACS",
    "B02001_007E": "sor_ACS",
    "B04006_022E": "brazilian_ACS",
    "B04006_045E": "guyanese_ACS",
    "B04006_074E": "cabo_verdean_ACS",
    "B04006_097E": "belizean_ACS", #fixed 
    "B04006_002E": "algerian_ACS",
    "B04006_004E": "armenian_ACS",
    "B04006_006E": "azerbaijani_ACS",
    "B04006_007E": "bahraini_ACS",
    "B04006_011E": "cypriot_ACS",
    "B04006_015E": "djiboutian_ACS",
    "B04006_028E": "egyptian_ACS",
    "B04006_032E": "georgian_ACS",
    "B04006_039E": "iranian_ACS",
    "B04006_040E": "iraqi_ACS",
    "B04006_041E": "israeli_ACS",
    "B04006_042E": "jordanian_ACS",
    "B04006_044E": "kuwaiti_ACS",
    "B04006_046E": "lebanese_ACS",
    "B04006_047E": "libyan_ACS",
    "B04006_048E": "maltese_ACS",
    "B04006_049E": "mauritanian_ACS",
    "B04006_050E": "moroccan_ACS",
    "B04006_052E": "omani_ACS",
    "B04006_054E": "palestinian_ACS",
    "B04006_055E": "qatari_ACS",
    "B04006_062E": "saudi_ACS",
    "B04006_077E": "sudanese_ACS",
    "B04006_078E": "syrian_ACS",
    "B04006_079E": "sahrawi_ACS",
    "B04006_080E": "tunisian_ACS",
    "B04006_088E": "turkish_ACS",
    "B04006_092E": "emirati_ACS",
    "B04006_093E": "yemeni_ACS"
}

mena_world_bank_ACS = [
    acs_vars["B04006_002E"],  # Algerian
    acs_vars["B04006_007E"],  # Bahraini
    acs_vars["B04006_015E"],  # Djiboutian
    acs_vars["B04006_028E"],  # Egyptian
    acs_vars["B04006_039E"],  # Iranian
    acs_vars["B04006_040E"],  # Iraqi
    acs_vars["B04006_041E"],  # Israeli
    acs_vars["B04006_042E"],  # Jordanian
    acs_vars["B04006_044E"],  # Kuwaiti
    acs_vars["B04006_046E"],  # Lebanese
    acs_vars["B04006_047E"],  # Libyan
    acs_vars["B04006_048E"],  # Maltese
    acs_vars["B04006_050E"],  # Moroccan
    acs_vars["B04006_052E"],  # Omani
    acs_vars["B04006_054E"],  # Palestinian
    acs_vars["B04006_055E"],  # Qatari
    acs_vars["B04006_062E"],  # Saudi
    acs_vars["B04006_078E"],  # Syrian
    acs_vars["B04006_080E"],  # Tunisian
    acs_vars["B04006_092E"],  # Emirati
    acs_vars["B04006_093E"],  # Yemeni
]


def fetch_data(base_url, var_map):
    params = {
        "get": ",".join(var_map.keys()),
        "for": "tract:*",
        "in": f"state:{state_fips} county:{county}",
        "key": API_KEY
    }
    r = requests.get(base_url, params=params)
    r.raise_for_status()
    data = r.json()
    df = pd.DataFrame(data[1:], columns=data[0])
    df["GEOID"] = df["state"] + df["county"] + df["tract"]
    df = df.rename(columns=var_map)
    return df

# Fetch PL and ACS data
pl_df = fetch_data("https://api.census.gov/data/2020/dec/pl", pl_vars)
acs_df = fetch_data("https://api.census.gov/data/2022/acs/acs5", acs_vars)

# Merge on GEOID
ma_merged_df = pd.merge(pl_df, acs_df, on="GEOID")

# Convert numeric columns
cols_to_numeric = list(pl_vars.values()) + list(acs_vars.values())
ma_merged_df[cols_to_numeric] = ma_merged_df[cols_to_numeric].apply(pd.to_numeric, errors="coerce")

# World Bank MENA
ma_merged_df["mena_world_bank_ACS"] = ma_merged_df[
    [
        "algerian_ACS", "bahraini_ACS", "djiboutian_ACS", "egyptian_ACS",
        "iranian_ACS", "iraqi_ACS", "israeli_ACS", "jordanian_ACS", "kuwaiti_ACS",
        "lebanese_ACS", "libyan_ACS", "maltese_ACS", "moroccan_ACS", "omani_ACS",
        "palestinian_ACS", "qatari_ACS", "saudi_ACS", "syrian_ACS",
        "tunisian_ACS", "emirati_ACS", "yemeni_ACS"
    ]
].sum(axis=1)
# UNHCR MENA
ma_merged_df["mena_unhcr_ACS"] = ma_merged_df[
    [
        "algerian_ACS", "bahraini_ACS", "egyptian_ACS", "iraqi_ACS", "jordanian_ACS",
        "kuwaiti_ACS", "lebanese_ACS", "libyan_ACS", "maltese_ACS", "mauritanian_ACS",
        "moroccan_ACS", "omani_ACS", "palestinian_ACS", "qatari_ACS", "saudi_ACS",
        "syrian_ACS", "tunisian_ACS", "emirati_ACS", "yemeni_ACS"
    ]
].sum(axis=1)
# UNSD MENA
ma_merged_df["mena_unsd_ACS"] = ma_merged_df[
    [
        "algerian_ACS", "armenian_ACS", "azerbaijani_ACS", "bahraini_ACS",
        "cypriot_ACS", "egyptian_ACS", "georgian_ACS", "iraqi_ACS", "israeli_ACS",
        "jordanian_ACS", "kuwaiti_ACS", "lebanese_ACS", "libyan_ACS", "moroccan_ACS",
        "omani_ACS", "palestinian_ACS", "qatari_ACS", "saudi_ACS", "sudanese_ACS",
        "syrian_ACS", "tunisian_ACS", "turkish_ACS", "emirati_ACS", "sahrawi_ACS",
        "yemeni_ACS"
    ]
].sum(axis=1)

print(ma_merged_df.head())


In [44]:


# Input your Census API key
API_KEY = "df8c76e570e47e69b0d4523fbde247bc091d81dc"

# Geographic scope: all tracts in Massachusetts
state_fips = "36"
county = "*"  # all counties
geography = "tract:*"

# Define variables to pull
pl_vars = {
    "P1_001N": "totpop_PL", #these have been checked
    "P2_002N": "hispanic_PL", 
    "P1_002N": "white_PL",
    "P1_004N": "black_PL",
    "P1_008N": "sor_PL"
}

acs_vars = {
    "B03002_001E": "totpop_ACS", #good
    "B03002_012E": "hispanic_ACS", #good
    "B02001_002E": "white_ACS", 
    "B02001_003E": "black_ACS",
    "B02001_007E": "sor_ACS",
    "B04006_022E": "brazilian_ACS",
    "B04006_045E": "guyanese_ACS",
    "B04006_074E": "cabo_verdean_ACS",
    "B04006_097E": "belizean_ACS" #fixed 
}

def fetch_data(base_url, var_map):
    params = {
        "get": ",".join(var_map.keys()),
        "for": "tract:*",
        "in": f"state:{state_fips} county:{county}",
        "key": API_KEY
    }
    r = requests.get(base_url, params=params)
    r.raise_for_status()
    data = r.json()
    df = pd.DataFrame(data[1:], columns=data[0])
    df["GEOID"] = df["state"] + df["county"] + df["tract"]
    df = df.rename(columns=var_map)
    return df

# Fetch PL and ACS data
pl_df = fetch_data("https://api.census.gov/data/2020/dec/pl", pl_vars)
acs_df = fetch_data("https://api.census.gov/data/2022/acs/acs5", acs_vars)

# Merge on GEOID
ny_merged_df = pd.merge(pl_df, acs_df, on="GEOID")

# Convert numeric columns
cols_to_numeric = list(pl_vars.values()) + list(acs_vars.values())
ny_merged_df[cols_to_numeric] = ny_merged_df[cols_to_numeric].apply(pd.to_numeric, errors="coerce")

print(ny_merged_df.head())


   totpop_PL  hispanic_PL  white_PL  black_PL  sor_PL state_x county_x  \
0       2073          258      1865      1279     149      36      001   
1       3125          345      2892      2027     144      36      001   
2       2598          413      2389      1898     198      36      001   
3       3190          524      2845      1919     223      36      001   
4       3496          380      3224       677     197      36      001   

  tract_x        GEOID  totpop_ACS  ...  white_ACS  black_ACS  sor_ACS  \
0  000100  36001000100        2259  ...        750        989      280   
1  000201  36001000201        2465  ...        388       1777       23   
2  000202  36001000202        2374  ...        317       1979        4   
3  000301  36001000301        2837  ...       1005       1271      236   
4  000302  36001000302        3200  ...       2039        565       45   

   brazilian_ACS  guyanese_ACS  cabo_verdean_ACS  belizean_ACS  state_y  \
0              0            13     